In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

In [14]:
######################## libraries ########################

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.ticker as ticker
import numpy as np

import sys
sys.path.append('../../')

from utils.utility_functions import label_top
from utils.utility_functions import get_last_name
from utils.utility_functions import make_dash_zero
# from utils.utility_functions import shorten_opera_name

##################### CHARTING OPTIONS #####################

# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['Palatino']})

rc('text', usetex=True)
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

colors = ['#8a170f', '#8a540f', '#828a0f', '#458a0f', '#0f8a17', '#0f8a54', '#0f828a', '#0f458a', '#170f8a', '#540f8a', '#8a0f82', '#8a0f45']


##################### SOURCES OPTIONS #####################

archival_sources = ['Metropolitan Opera', 'Opera Scotland', 'Royal Opera House', 'Teatro alla Scala', 'Wiener Staatsoper', 
                    'Chicago Lyric Opera','San Francisco Opera','Czech National Theatre','Glyndebourne','Salzburg Festival',
                    'Sweden Opera House','National Opera de Paris','Opera Comique']
commercial_sources_world = ['bachtrack.com','operabase.com','operacritic.com']
commercial_sources_uk = ['Concert-Diary','U.K. Theatre Web']
government_sources = ['National Central Opera Service Reports', 'Opera America']

##############################################################################

In [66]:
import pandas as pd

columns=['source_id', 'source_name', 'composer', 'work', 'country', 'state', 'city', 'theatre', 'date_start', 'date_end', 'premiere_date', 'performance_count']

cursor = conn.cursor()  
query = "SELECT s.id as source_id, " + \
        "       s.name as source_name, " + \
        "       coalesce(c.name, l.composer) as composer, " + \
        "       coalesce(w.name, l.work) as work, " + \
        "       l.country, " + \
        "       l.state, " + \
        "       l.city, " + \
        "       l.theatre, " + \
        "       l.date_start, " + \
        "       l.date_end, " + \
        "       w.premiere_date, " + \
        "       l.performance_count " + \
        "from sources s " + \
        "INNER JOIN listings l on s.id = l.source_id " + \
        "LEFT JOIN listing_mappers lm ON l.map_key = lm.map_key " + \
        "LEFT JOIN works w ON lm.work_uri = w.uri " + \
        "LEFT JOIN composers c ON w.composer_id = c.id " + \
        "ORDER BY s.id, l.country, l.state, l.city"
        
cursor.execute(query)

mdf = pd.DataFrame(cursor.fetchall(), columns=columns)
mdf['year'] = mdf.apply(lambda row: row['date_start'].year, axis=1)
mdf['decade'] = mdf.apply(lambda row: 10*int(row['year']/10), axis=1)
mdf[['premiere_date']] = mdf[['premiere_date']].fillna(value=0)
mdf['work_decade'] = mdf.apply(lambda row: 10*int(row['premiere_date']/10), axis=1)

In [28]:
######################## top operas ########################
sources = archival_sources

o_df = mdf.copy()
o_df = o_df[o_df['source_name'].isin(sources)]
o_df = o_df[o_df['composer'] != 'Unknown']
o_df = o_df[o_df['composer'] != '']

for num, source in enumerate(sources):
    c_df = o_df[o_df['source_name']==source]
    c_df = c_df.groupby(['work'])
    c_df = c_df.agg({'performance_count': np.sum })
    c_df = c_df.sort_values(by=['performance_count'], ascending=False)    
    c_df = c_df.head(5)
    c_df = c_df.reset_index(level=['work'])  
    c_df[source] = c_df['work']    
    c_df = c_df[source]
    if num==0:
        df = c_df.copy().to_frame()
    else:
        df = df.join(c_df, how='outer')
    
df = df.T

for idx,row in df.iterrows():
    values = row.values.tolist()
    values = [shorten_opera_name(c) for c in values ]
    print(idx + ' & ' + ' & '.join(values) + " \\\\")
###############################################################

Metropolitan Opera & La bohème & Aida & Carmen & Traviata & Tosca \\
Opera Scotland & La bohème & M. Figaro & Traviata & M. Flute & Butterfly \\
Royal Opera House & Tosca & La bohème & Traviata & Turandot & Rigoletto \\
Teatro alla Scala & La bohème & Butterfly & Aida & Rigoletto & Barber of Svl \\
Wiener Staatsoper & Tosca & M. Figaro & M. Flute & D. Rosenkav. & Les Huguenots \\
Chicago Lyric Opera & La bohème & Butterfly & Tosca & Traviata & Carmen \\
San Francisco Opera & La bohème & Butterfly & Carmen & Tosca & Traviata \\
Czech National Theatre & Bart. Bride & Rusalka & The Kiss & Don Giovanni & Carmen \\
Glyndebourne & M. Figaro & Così & Don Giovanni & M. Flute & Abd. Serail \\
Salzburg Festival & M. Figaro & M. Flute & Don Giovanni & Così & Abd. Serail \\
Sweden Opera House & Carmen & M. Figaro & Barber of Svl & La bohème & M. Flute \\
National Opera de Paris & M. Figaro & M. Flute & La bohème & Tosca & Carmen \\
Opera Comique & Carmen & Faust & Barber of Svl & Manon & Mignon \\

In [57]:
######################## top composers table ########################

sources = archival_sources

o_df = mdf.copy()
o_df = o_df[o_df['source_name'].isin(sources)]
o_df = o_df[o_df['composer'] != 'Unknown']
o_df = o_df[o_df['composer'] != '']

top_composers = []

for num, source in enumerate(sources):
    c_df = o_df[o_df['source_name']==source]
    c_df = c_df.groupby(['work'])
    c_df = c_df.agg({'performance_count': np.sum })
    c_df = c_df.sort_values(by=['performance_count'], ascending=False)    
    top = c_df.head(5)
    for composer in list(top.index):
        if composer not in top_composers:
            top_composers.append(composer) 
    c_df = c_df.reset_index(level=['work'])  
    c_df.index += 1
    c_df = c_df.reset_index() 
    c_df = c_df.rename({'index': source}, axis='columns')
    c_df = c_df.set_index('work')
    c_df = c_df[[source]]
    if num==0:
        df = c_df.copy()
    else:
        df = df.join(c_df, how='outer')

df = df.reset_index(level=['work'])  
df = df[df['work'].isin(top_composers)]  
df = df.fillna(value=500)

decades = df.columns.get_values().tolist()
decades = [str(d) for d in decades[1:]]
o_df = o_df.fillna(value=0)
df['rank'] = df.apply(lambda row: sum(row.values.tolist()[1:]), axis=1)
df = df.sort_values(by='rank')
df = df.drop(['rank'], axis=1)

print('r'*len(decades))
print('\\textbf{Opera} & \\textbf{' + ' & \\textbf{'.join(decades) + ' \\\ \midrule ')

for idx,row in df.iterrows():
    values = row.values.tolist()
    values = [0 if c==500 else c for c in values[1:] ]    
    values = [make_dash_zero(c) for c in values ]
    print(row['work'] + ' & ' + ' & '.join(values) + " \\\\")

rrrrrrrrrrrrr
\textbf{Opera} & \textbf{Metropolitan Opera & \textbf{Opera Scotland & \textbf{Royal Opera House & \textbf{Teatro alla Scala & \textbf{Wiener Staatsoper & \textbf{Chicago Lyric Opera & \textbf{San Francisco Opera & \textbf{Czech National Theatre & \textbf{Glyndebourne & \textbf{Salzburg Festival & \textbf{Sweden Opera House & \textbf{National Opera de Paris & \textbf{Opera Comique \\ \midrule 
Carmen & 3 & 8 & 11 & 12 & 13 & 5 & 3 & 5 & 18 & 17 & 1 & 5 & 1 \\
La bohème & 1 & 1 & 2 & 1 & 8 & 1 & 1 & 17 & 7 & 49 & 4 & 3 & 33 \\
The Magic Flute & 19 & 4 & 8 & 35 & 3 & 12 & 14 & 14 & 4 & 2 & 5 & 2 & 9 \\
The Barber of Seville & 10 & 9 & 26 & 5 & 21 & 7 & 8 & 12 & 12 & 13 & 3 & 7 & 3 \\
Don Giovanni & 13 & 7 & 12 & 15 & 22 & 15 & 9 & 4 & 3 & 3 & 8 & 6 & 31 \\
La traviata & 4 & 3 & 3 & 10 & 19 & 4 & 5 & 9 & 16 & 38 & 7 & 12 & 43 \\
The Marriage of Figaro & 16 & 2 & 6 & 17 & 2 & 11 & 12 & 8 & 1 & 1 & 2 & 1 & 163 \\
Così fan tutte & 41 & 6 & 14 & 24 & 32 & 13 & 19 & 25 & 2 & 4 & 

In [68]:
######################## Composer works in theatre chart ########################

o_df = mdf.copy()
o_df = o_df[o_df['source_name']=='Metropolitan Opera']
o_df = o_df[o_df['composer'] == 'Giuseppe Verdi']

works = list(o_df['work'].unique())
num_works = len(works)

o_df = o_df.groupby(['work', 'decade'])
o_df = o_df.agg({'performance_count': np.sum })
o_df = o_df.reset_index(level=['work', 'decade'])

o_df = o_df.pivot(index='work', columns='decade', values=['performance_count'])
o_df = o_df.fillna(value=0)

decades = o_df.columns.get_level_values(1).get_values().tolist()
decades = [str(d) for d in decades]

print('r'*len(decades))
print('\\textbf{Opera} & \\textbf{' + ' & \\textbf{'.join(decades) + ' \\\ \midrule ')

for idx,row in o_df.iterrows():
    values = row.values.tolist()
    values = [make_dash_zero(c) for c in values ]
    print(idx + ' & ' + ' & '.join(values) + " \\\\")

rrrrrrrrrrrrrr
\textbf{Opera} & \textbf{1880 & \textbf{1890 & \textbf{1900 & \textbf{1910 & \textbf{1920 & \textbf{1930 & \textbf{1940 & \textbf{1950 & \textbf{1960 & \textbf{1970 & \textbf{1980 & \textbf{1990 & \textbf{2000 & \textbf{2010 \\ \midrule 
Aida & 7 & 40 & 95 & 108 & 100 & 87 & 79 & 117 & 126 & 109 & 65 & 92 & 67 & 50 \\
Don Carlos & - & - & - & - & 14 & - & - & 38 & 28 & 28 & 40 & 14 & 23 & 27 \\
Ernani & - & - & 4 & - & 19 & - & - & 8 & 12 & 3 & 32 & - & 7 & 13 \\
Falstaff & - & 22 & 5 & 5 & 22 & 5 & 7 & - & 41 & 22 & 14 & 15 & 17 & 10 \\
I Lombardi alla prima crociata & - & - & - & - & - & - & - & - & - & - & - & 11 & - & - \\
I vespri siciliani & - & - & - & - & - & - & - & - & 1 & 28 & 8 & - & 8 & - \\
Il trovatore & 15 & 25 & 36 & 56 & 38 & 35 & 41 & 61 & 83 & 65 & 74 & 41 & 43 & 41 \\
Inno delle Nazioni & - & - & - & - & - & - & - & - & 1 & - & - & - & - & - \\
La forza del destino & - & - & - & 11 & 23 & 12 & 11 & 40 & 50 & 40 & 28 & 10 & 9 & - \\
La traviata & 6 & 

In [56]:
df

,work,Metropolitan Opera,Opera Scotland,Royal Opera House,Teatro alla Scala,Wiener Staatsoper,Chicago Lyric Opera,San Francisco Opera,Czech National Theatre,Glyndebourne,Salzburg Festival,Sweden Opera House,National Opera de Paris,Opera Comique
208,Carmen,3.0,8.0,11.0,12.0,13.0,5.0,3.0,5.0,18.0,17.0,1.0,5.0,1.0
1100,La bohème,1.0,1.0,2.0,1.0,8.0,1.0,1.0,17.0,7.0,49.0,4.0,3.0,33.0
2027,The Magic Flute,19.0,4.0,8.0,35.0,3.0,12.0,14.0,14.0,4.0,2.0,5.0,2.0,9.0
1934,The Barber of Seville,10.0,9.0,26.0,5.0,21.0,7.0,8.0,12.0,12.0,13.0,3.0,7.0,3.0
436,Don Giovanni,13.0,7.0,12.0,15.0,22.0,15.0,9.0,4.0,3.0,3.0,8.0,6.0,31.0
1158,La traviata,4.0,3.0,3.0,10.0,19.0,4.0,5.0,9.0,16.0,38.0,7.0,12.0,43.0
2035,The Marriage of Figaro,16.0,2.0,6.0,17.0,2.0,11.0,12.0,8.0,1.0,1.0,2.0,1.0,163.0
255,Così fan tutte,41.0,6.0,14.0,24.0,32.0,13.0,19.0,25.0,2.0,4.0,27.0,11.0,47.0
1446,Madama Butterfly,7.0,5.0,9.0,2.0,12.0,2.0,2.0,16.0,81.0,500.0,9.0,10.0,113.0
2131,Tosca,5.0,10.0,1.0,6.0,1.0,3.0,4.0,20.0,500.0,78.0,6.0,4.0,142.0


In [56]:
import pandas as pd

columns=['source_id', 'source_name', 'composer', 'work', 'country', 'state', 'city', 'theatre', 'date_start', 'date_end', 'premiere_date', 'performance_count', 'theatre_category']

cursor = conn.cursor()  
query = "SELECT s.id as source_id, " + \
        "       s.name as source_name, " + \
        "       coalesce(c.name, l.composer) as composer, " + \
        "       coalesce(w.name, l.work) as work, " + \
        "       l.country, " + \
        "       l.state, " + \
        "       l.city, " + \
        "       l.theatre, " + \
        "       l.date_start, " + \
        "       l.date_end, " + \
        "       w.premiere_date, " + \
        "       l.performance_count, " + \
        "       CASE WHEN l.source_id = 1 then 'Professional' when opera_america_name IS NULL THEN 'Amateur' ELSE 'Professional' END as theatre_category " + \
        "from sources s " + \
        "INNER JOIN listings l on s.id = l.source_id " + \
        "LEFT JOIN theatre_mapping tm ON l.theatre = tm.name " + \
        "LEFT JOIN listing_mappers lm ON l.map_key = lm.map_key " + \
        "LEFT JOIN works w ON lm.work_uri = w.uri " + \
        "LEFT JOIN composers c ON w.composer_id = c.id " + \
        "WHERE source_id IN (1, 6) AND w.uri != 'W1' " + \
        "ORDER BY s.id, l.country, l.state, l.city"
        
cursor.execute(query)

mdf = pd.DataFrame(cursor.fetchall(), columns=columns)
mdf['year'] = mdf.apply(lambda row: row['date_start'].year, axis=1)
mdf['decade'] = mdf.apply(lambda row: 10*int(row['year']/10), axis=1)
mdf[['premiere_date']] = mdf[['premiere_date']].fillna(value=0)
mdf['work_decade'] = mdf.apply(lambda row: 10*int(row['premiere_date']/10), axis=1)

mdf.loc[mdf['theatre']=='New York City Opera', 'theatre_category'] = 'Professional'

In [61]:
######################## top composers over decades ########################
print(government_sources)

o_df = mdf.copy()
# o_df = o_df[o_df['source_name']=='National Central Opera Service Reports']
o_df = o_df[o_df['theatre_category']=='Amateur']
o_df = o_df[o_df['composer'] != 'Unknown']
o_df = o_df[o_df['composer'] != 'nan']
o_df = o_df[o_df['composer'] != '']
o_df = o_df[o_df['decade'] >= 1770]
o_df = o_df[o_df['year'] < 2019]


decades = list(o_df['decade'].unique())
decades.sort()

top_composers = []

for num, decade in enumerate(decades):
    c_df = o_df[o_df['decade']==decade]
    c_df = c_df.groupby(['composer'])
    c_df = c_df.agg({'performance_count': np.sum })
    c_df = c_df.sort_values(by=['performance_count'], ascending=False)    
    top = c_df.head(10)
#     print(top)
    for composer in list(top.index):
        if composer not in top_composers:
            top_composers.append(composer) 
    c_df = c_df.reset_index(level=['composer'])  
    c_df.index += 1
    c_df = c_df.reset_index() 
    c_df = c_df.rename({'index': decade}, axis='columns')
    c_df = c_df.set_index('composer')
    c_df = c_df[[decade]]
    if num==0:
        df = c_df.copy()
    else:
        df = df.join(c_df, how='outer')
        
df = df.reset_index(level=['composer'])  
df = df[df['composer'].isin(top_composers)] 
df.loc[df['composer']=='Franz Lehár', 'composer'] = 'Franz Lehar'
df = df.sort_values(by=[decades[-1]], na_position='last')    
df = df.fillna(value=0)

decades = df.columns.get_values().tolist()
decades = [str(d) for d in decades[1:]]

print('r'*len(decades))
print('\\textbf{Composer} & \\textbf{' + ' & \\textbf{'.join(decades) + ' \\\ \midrule ')

for idx,row in df.iterrows():
    values = row.values.tolist()
    values = [make_dash_zero(c) for c in values[1:] ]
    print(get_last_name(row['composer']) + ' & ' + ' & '.join(values) + " \\\\")

# print(df)

['National Central Opera Service Reports', 'Opera America']
rrrrr
\textbf{Composer} & \textbf{1950 & \textbf{1960 & \textbf{1970 & \textbf{1980 & \textbf{1990 \\ \midrule 
Sullivan & 15 & 7 & 2 & 1 & 1 \\
Puccini & 2 & 2 & 1 & 3 & 2 \\
Mozart & 7 & 1 & 3 & 2 & 3 \\
Sondheim & - & 87 & 55 & 10 & 4 \\
Verdi & 4 & 3 & 4 & 4 & 5 \\
Weill & 12 & 19 & 16 & 12 & 6 \\
Bernstein & 11 & 34 & 20 & 16 & 7 \\
Lehar & - & 30 & 13 & 19 & 8 \\
Webber & - & - & 131 & 34 & 9 \\
Gershwin & - & 67 & 50 & 17 & 10 \\
Rossini & 8 & 5 & 7 & 8 & 11 \\
Rodgers & 9 & 64 & 25 & 9 & 16 \\
Donizetti & 19 & 6 & 6 & 6 & 17 \\
Strauss II & 25 & 9 & 9 & 14 & 22 \\
Bizet & - & 8 & 11 & 20 & 29 \\
Menotti & 1 & 4 & 5 & 5 & 37 \\
Offenbach & - & 15 & 10 & 11 & 43 \\
Humperdinck & 6 & 10 & 8 & 7 & 98 \\
Wolf-Ferrari & 5 & 39 & 53 & 87 & 131 \\
Floyd & 3 & 37 & 24 & 60 & - \\
Thompson & 10 & 157 & 258 & 223 & - \\


In [69]:
import pandas as pd

columns=['source_id', 'source_name', 'composer', 'work', 'country', 'state', 'city', 'theatre', 
         'date_start', 'date_end', 'premiere_date', 'performance_count', 'composer_country']

cursor = conn.cursor()  
query = "SELECT s.id as source_id, " + \
        "       s.name as source_name, " + \
        "       coalesce(c.name, l.composer) as composer, " + \
        "       coalesce(w.name, l.work) as work, " + \
        "       l.country, " + \
        "       l.state, " + \
        "       l.city, " + \
        "       l.theatre, " + \
        "       l.date_start, " + \
        "       l.date_end, " + \
        "       w.premiere_date, " + \
        "       l.performance_count, " + \
        "       c.country as composer_country " + \
        "from sources s " + \
        "INNER JOIN listings l on s.id = l.source_id " + \
        "LEFT JOIN listing_mappers lm ON l.map_key = lm.map_key " + \
        "LEFT JOIN works w ON lm.work_uri = w.uri " + \
        "LEFT JOIN composers c ON w.composer_id = c.id " + \
        "WHERE source_id IN (4, 8, 13) AND w.uri != 'W1' " + \
        "ORDER BY s.id DESC, l.country, l.state, l.city"
        
cursor.execute(query)

mdf = pd.DataFrame(cursor.fetchall(), columns=columns)
mdf['year'] = mdf.apply(lambda row: row['date_start'].year, axis=1)
mdf['decade'] = mdf.apply(lambda row: 10*int(row['year']/10), axis=1)
mdf[['premiere_date']] = mdf[['premiere_date']].fillna(value=0)
mdf['work_decade'] = mdf.apply(lambda row: 10*int(row['premiere_date']/10), axis=1)
mdf = mdf[mdf['year'] > 2007]
print(mdf.shape)
mdf = mdf[['composer', 'work', 'country', 'state', 'city', 'date_start', 'date_end', 'premiere_date', 'performance_count', 'year', 'decade', 'work_decade', 'composer_country']]
mdf = mdf.drop_duplicates()
print(mdf.shape)

(250428, 16)
(171608, 13)


In [85]:
def shorten_opera_name(text):
    try:
        return works_dict[text]
    except:
        return text
        
works_dict = {
    'La traviata': 'Traviata',
    'Madama Butterfly': 'Butterfly',
    'La damnation de Faust': 'Faust',
    'The Barber of Seville': 'Barber of Svl',
    'The Marriage of Figaro': 'M. Figaro',
    'The Magic Flute': 'M. Flute',
    'Così fan tutte': 'Così',
    'Don Giovanni': 'Don Giovanni',
    'Der Rosenkavalier': 'Der Rosenkavalier',
    'The Enchanted Island': 'The E. Island',
     "L'elisir d'amore":  "L'elisir",
    'Cavalleria rusticana': 'Cavalleria',
    'The Flying Dutchman': 'The F. Dutchman',
    'The Bartered Bride': 'Bart. Bride',
    'Die Entführung aus dem Serail': 'Abd. Serail',
    'Der Rosenkavalier': 'D. Rosenkav.',
    'Little Red Riding Hood': 'L.R. Riding Hood',
    'Hexe Hillary geht in die Oper': 'Hexe Hillary',
    "Les contes d'Hoffmann": 'Hoffmann',
    'Beware! The Prince is Messy': 'Prince is messy',
    'The Nightingale of Gorenjska': 'N. of Gorenjska',
    'Ero s onoga svijeta': 'Ero',
    'Le notti delle streghe': 'Le notti delle s.',
    'Postcard from Morocco': 'P. from Morocco',
    'Lucia di Lammermoor': 'Lucia di L.',    
    'The Flying Dutchman': 'The F. Dutchman',
    "L'enfant et les sortilèges": "L'enfant et les s.", 
    "Sweeney Todd: The Demon Barber of Fleet Street": 'Sweeney Todd',
    'The Merry Widow': 'Merry Widow'
}

In [86]:
######################## top works by country ########################

o_df = mdf.copy()
o_df = o_df[o_df['composer'] != 'Unknown']
o_df = o_df[o_df['composer'] != '']

countries_excl = ['Oman','Macedonia','Armenia','South Korea','Egypt','Georgia','Albania','Kyrgyzstan','Mongolia','Singapore',
                  'Bosnia-Herzegovina','Malta','Iceland','Mexico','China','Thailand','Peru','Costa Rica','India','UAE']

o_df = o_df[~o_df['country'].isin(countries_excl)]
countries = o_df['country'].unique().tolist()
countries.sort()

for num, country in enumerate(countries):
    c_df = o_df[o_df['country']==country]
    c_df = c_df.groupby(['work'])
    c_df = c_df.agg({'performance_count': np.sum })
    c_df = c_df.sort_values(by=['performance_count'], ascending=False)    
    c_df = c_df.head(5)
    c_df = c_df.reset_index(level=['work'])  
    c_df.index += 1    
    c_df = c_df.rename({'index': 'Ranking'}, axis='columns')
    c_df = c_df.rename({'work': country}, axis='columns')    
#     print(c_df.head())
#     c_df = c_df.set_index('Ranking')
    c_df = c_df[country]
    
    if num==0:
        df = c_df.copy().to_frame()
    else:
        df = df.join(c_df, how='outer')
        
df = df.T
df      

#For displaying as rows
decades = df.columns.get_values().tolist()
decades = [str(d) for d in decades]

print('l'*len(decades))
print('\\textbf{Country} & \\textbf{' + ' & \\textbf{'.join(decades) + ' \\\ \midrule ')    
for idx,row in df.iterrows():
    values = row.values.tolist()
    values = [shorten_opera_name(c) for c in values ]    
    print(str(idx) + ' & ' + ' & '.join(values) + " \\\\")

# for idx,row in df.iterrows():
#     values = row.values.tolist()
#     values = [shorten_opera_name(c) for c in values ]
#     print(idx + ' & ' + ' & '.join(values) + " \\\\")
# ###############################################################

lllll
\textbf{Country} & \textbf{1 & \textbf{2 & \textbf{3 & \textbf{4 & \textbf{5 \\ \midrule 
Argentina & Traviata & La bohème & Così & Werther & Tosca \\
Australia & La bohème & Butterfly & Carmen & My Fair Lady & Traviata \\
Austria & M. Flute & Carmen & Die Fledermaus & Countess Maritza & Traviata \\
Azerbaijan & Leyli and Majnun & Traviata & Asli and Kerem & Il tabarro & Shah Ismayil \\
Belarus & Carmen & Eugene Onegin & Traviata & Iolanta & Aida \\
Belgium & M. Flute & Otello & Don Giovanni & Carmen & La bohème \\
Bulgaria & Traviata & Countess Maritza & Tosca & Aida & Butterfly \\
Canada & La bohème & Tosca & Barber of Svl & Butterfly & M. Figaro \\
Chile & Don Giovanni & Barber of Svl & Aida & M. Flute & Rigoletto \\
Croatia & Ero & Nikola Šubić Zrinski & Hexe Hillary & La bohème & Traviata \\
Czech Republic & Rusalka & Traviata & Carmen & Don Giovanni & M. Flute \\
Denmark & La bohème & Traviata & Tosca & M. Figaro & Carmen \\
Estonia & Countess Maritza & Carmen & Ball at the